In [ ]:
# Author: [Daehee Yang]
# Purpose: Modularized NMF-ICA spectrum decomposition using Hyperspy
# Source: Refactored from original workflow

%matplotlib qt

import hyperspy.api as hs # noqa
import numpy as np # noqa
import matplotlib.pyplot as plt # noqa
from sklearn.cluster import KMeans # noqa
from sklearn.preprocessing import MinMaxScaler # noqa
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist

def remove_negative(signal):
    """Ensure spectrum has no negative values."""
    corrected = signal.data - np.min(signal.data)
    new_signal = hs.signals.EELSSpectrum(corrected)
    for i in range(3):
        new_signal.axes_manager[i].size = signal.axes_manager[i].size
        new_signal.axes_manager[i].scale = signal.axes_manager[i].scale
        new_signal.axes_manager[i].offset = signal.axes_manager[i].offset
        new_signal.axes_manager[i].units = signal.axes_manager[i].units
    return new_signal

def run_nmf(signal, n_components=4, max_iter=500000):
    """Run NMF decomposition on signal."""
    signal.decomposition(
        normalize_poissonian_noise=True,
        algorithm="NMF",
        output_dimension=n_components,
        max_iter=max_iter
    )

def run_ica(signal, n_components=4, max_iter=50000):
    """Apply ICA to decompose sources after NMF."""
    signal.blind_source_separation(
        number_of_components=n_components,
        algorithm='sklearn_fastica',
        diff_order=1,
        reverse_component_criterion="loadings",
        max_iter=max_iter
    )

def segment_by_kmeans(signal, n_clusters):
    """
    Perform K-means clustering on a 2D Hyperspy signal and return:
    - Binary segmentation mask
    - Silhouette Coefficient
    - Dunn Index

    Parameters:
        signal (hs.signals.Signal2D): Input signal to segment.
        n_clusters (int): Number of clusters to form.

    Returns:
        segmented (hs.signals.Signal2D): Binary segmentation mask.
        silhouette (float): Silhouette Coefficient.
        dunn (float): Dunn Index.
    """
    # Reshape and normalize
    data = signal.data
    h, w = data.shape
    reshaped = data.reshape(-1, 1)
    scaled = MinMaxScaler().fit_transform(reshaped)

    # K-means clustering
    model = KMeans(n_clusters=n_clusters, random_state=0, n_init=50)
    model.fit(scaled)
    labels = model.labels_.reshape(h, w)

    # Binary segmentation: background = lowest mean intensity cluster
    intensities = [data[labels == i].mean() for i in range(n_clusters)]
    background_idx = int(np.argmin(intensities))
    binary = (labels != background_idx).astype(np.uint8) * 255
    segmented = hs.signals.Signal2D(binary)
    segmented.change_dtype("uint16")

    return segmented

In [2]:
# =====================
# Example Usage
# =====================

# Set parameters
path = 'E:/DaeheeYang/data/2024-02-25_MEA-HM-ref-EOL-sulfur/analysis/SI data (2)'
filename = 'EELS Spectrum Image (high-loss) (aligned) (aligned)_Cleaned'
save_name = 'decompose'
ext = 'dm4'

crop_range = (280.0, 300.0)
n_components = 4

# Load and preprocess spectrum
a = hs.load(f'{path}/{filename}.{ext}')
s = a.isig[crop_range[0] : crop_range[1]]
s = remove_negative(s)
s.metadata = a.metadata
#s.plot()

c:\Users\Yang\miniconda3\envs\venv_py38\lib\site-packages\hyperspy\signal.py:2450: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(


In [3]:
# Run decomposition
run_nmf(s, n_components)
run_ica(s, n_components)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=4
  centre=None
scikit-learn estimator:
NMF(max_iter=500000, n_components=4)
[########################################] | 100% Completed | 117.03 ms


Blind source separation info:
  number_of_components=4
  algorithm=sklearn_fastica
  diff_order=1
  reverse_component_criterion=loadings
  whiten_method=PCA
scikit-learn estimator:
FastICA(max_iter=50000, tol=1e-10, whiten=False)


c:\Users\Yang\miniconda3\envs\venv_py38\lib\site-packages\sklearn\decomposition\_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


In [4]:
#If you want to check the results, please run the code below
s.plot_decomposition_loadings()
s.plot_decomposition_factors()
s.plot_decomposition_results()
s.plot_bss_loadings()
s.plot_bss_factors()
s.plot_bss_results()

In [ ]:
# Save the result of decomposition
s.export_bss_results(folder=f'{path}/{save_name}',
                    factor_format='MSA',
                    loading_format='TIFF')
s.export_decomposition_results(folder=f'{path}/{save_name}',
                            factor_format='MSA',
                            loading_format='TIFF')

In [52]:
# Load and segment coefficient maps
ion_data = hs.load(f"{path}/ionomer.{ext}")
ionomer_seg = segment_by_kmeans(ion_data, n_clusters=4)
ionomer_seg.plot(title='ionomer')

sig_data = hs.load(f"{path}/sigma.{ext}")
sigma_seg = segment_by_kmeans(sig_data, n_clusters=3)
sigma_seg.plot(title='sigma')

pi_data = hs.load(f"{path}/pi.{ext}")
pi_seg = segment_by_kmeans(pi_data, n_clusters=3)
pi_seg.plot(title='pi')

In [10]:
# Load and segment coefficient maps with metric output
ion_data = hs.load(f"{path}/ionomer.{ext}")
ionomer_seg, ion_sil, ion_dunn = segment_by_kmeans(ion_data, n_clusters=4)
ionomer_seg.plot(title=f'ionomer\nSilhouette={ion_sil:.3f}, Dunn={ion_dunn:.3f}')

sig_data = hs.load(f"{path}/sigma.{ext}")
sigma_seg, sig_sil, sig_dunn = segment_by_kmeans(sig_data, n_clusters=4)
sigma_seg.plot(title=f'sigma\nSilhouette={sig_sil:.3f}, Dunn={sig_dunn:.3f}')

pi_data = hs.load(f"{path}/pi.{ext}")
pi_seg, pi_sil, pi_dunn = segment_by_kmeans(pi_data, n_clusters=4)
pi_seg.plot(title=f'pi\nSilhouette={pi_sil:.3f}, Dunn={pi_dunn:.3f}')

In [54]:
# Support and total area maps
support_area = ((pi_seg.data + sigma_seg.data) > 0).astype(np.uint8) * 255
support_area = hs.signals.Signal2D(support_area)
support_area.change_dtype("uint16")

total_area = ((ionomer_seg.data + pi_seg.data + sigma_seg.data) > 0).astype(np.uint8) * 255
total_area = hs.signals.Signal2D(total_area)
total_area.change_dtype("uint16")

# Combined area logic and statistics
area_seg = (ionomer_seg.data * 3 + support_area.data * 2)
area_seg = hs.signals.Signal2D(area_seg)

total_count = (total_area.data == 255).sum()
both_count = np.sum(area_seg.data == 255 * 5)
only_ionomer = np.sum(area_seg.data == 255 * 3)
only_support = np.sum(area_seg.data == 255 * 2)

print(f"Total area : {total_count}")
print(f"Ionomer + support : {both_count}")
print(f"Only ionomer : {only_ionomer}")
print(f"Only support : {only_support}")

Total area : 5026
Ionomer + support : 3912
Only ionomer : 293
Only support : 821


In [55]:
# Save segmentation maps
ionomer_seg.save(f"{path}/seg_map/ionomer_seg_{(ionomer_seg.data == 255).sum()}.tif")
sigma_seg.save(f"{path}/seg_map/sigma_seg_{(sigma_seg.data == 255).sum()}.tif")
pi_seg.save(f"{path}/seg_map/pi_seg_{(pi_seg.data == 255).sum()}.tif")
support_area.save(f"{path}/seg_map/support_map_{(support_area.data == 255).sum()}.tif")
total_area.save(f"{path}/seg_map/total_map_{(total_area.data == 255).sum()}.tif")

In [56]:
# Save statistics
with open(f"{path}/seg_map/Statistical_analysis.txt", 'w') as txt:
    txt.writelines([
        f"Total area : {total_count}\n",
        f"Ionomer + support : {both_count}\n",
        f"Only ionomer : {only_ionomer}\n",
        f"Only support : {only_support}\n"
    ])